In [28]:

import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import sqlite3
import gradio as gr
import json

In [29]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4.1-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2" - example model name for Ollama - seems to small for this task
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [30]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
You can update the prices if asked - 
"""

## Database Setup

In [31]:
DB = "prices.db"

# Create the database and prices table if it doesn't exist
with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

## Tool Functions
These Python functions get executed when the LLM calls them.

In [32]:
def get_ticket_price(city):
    """
    🔧 EXECUTED WHEN TOOL IS CALLED
    Retrieves the price for a specific city from the database.
    """
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [33]:
def set_ticket_price(city, price):
    """
    🔧 EXECUTED WHEN TOOL IS CALLED
    Updates or inserts a ticket price for a specific city in the database.
    """
    print(f"DATABASE TOOL CALLED: Setting price for {city} to ${price}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        # UPSERT: Insert or update if city already exists
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', 
                      (city.lower(), price, price))
        conn.commit()

## Tool Dispatch Configuration
This maps tool names to functions and defines how to handle responses.

In [34]:


# Define the actual Python functions (assuming they exist elsewhere)
# def get_ticket_price(city): ...
# def set_ticket_price(city, price): ...

# Map LLM tool names to Python functions and how to build the response
TOOL_DISPATCH = {
    "get_ticket_price": {
        "func": get_ticket_price,
        "args": ["destination_city"], # Keys to extract from LLM arguments
        "response_template": lambda city, result: result # Simple content is the result
    },
    "set_ticket_price": {
        "func": set_ticket_price,
        "args": ["destination_city", "price"],
        "response_template": lambda city, price, result: f"Price for {city} updated to ${price}"
    }
}

# Function schema for getting ticket prices
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

# Function schema for setting ticket prices
set_price_function = {
    "name": "set_ticket_price",
    "description": "Set the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "number",
                "description": "The new price for the ticket",
            },
        },
        "required": ["destination_city", "price"],
        "additionalProperties": False
    }
}



def handle_tool_calls_improved(message):
    responses = []
    
    # 1. Iterate through all tool calls from the LLM
    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        
        # 2. Check if the tool is registered in our dispatch dictionary
        if tool_name not in TOOL_DISPATCH:
            # Handle unknown tool gracefully
            responses.append({
                "role": "tool",
                "content": f"Error: Unknown tool '{tool_name}'.",
                "tool_call_id": tool_call.id
            })
            continue

        tool_config = TOOL_DISPATCH[tool_name]
        
        # 3. Parse arguments once
        arguments = json.loads(tool_call.function.arguments)
        
        # 4. Prepare arguments for the function call
        try:
            # Extract only the required arguments from the LLM's payload
            func_args = [arguments[key] for key in tool_config["args"]]
        except KeyError as e:
            # Handle missing argument gracefully
            responses.append({
                "role": "tool",
                "content": f"Error: Tool '{tool_name}' missing required argument: {e}",
                "tool_call_id": tool_call.id
            })
            continue
            
        # 5. Call the actual Python function
        result = tool_config["func"](*func_args)
        
        # 6. Generate the content based on the response_template
        # (Pass original arguments and the result for template use)
        response_content = tool_config["response_template"](*func_args, result=result)
        
        # 7. Append the final response object
        responses.append({
            "role": "tool",
            "content": response_content,
            "tool_call_id": tool_call.id
        })
        
    return responses

In [35]:
# Combine both function definitions into the tools list
tools = [
    {"type": "function", "function": price_function},
    {"type": "function", "function": set_price_function}
]

In [36]:
def chat(message, history):
    """
    Main chat function that handles the conversation and tool calling loop.
    """
    # Convert Gradio history format to OpenAI message format
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    
    # Build the full message list: system message + history + new user message
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # First API call - LLM decides what to do
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    print(response.choices[0])
    
    # 🔄 TOOL CALLING LOOP
    # Keep looping while the LLM wants to call functions
    while response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        
        # 🔧 EXECUTE THE TOOLS - This is where our Python functions run!
        responses = handle_tool_calls_improved(message)
        
        # Add the tool call request and results to the conversation
        messages.append(message)
        messages.extend(responses)
        
        # Call the LLM again with the tool results
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    # Return the final response to the user
    return response.choices[0].message.content

In [37]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7893
* To create a public link, set `share=True` in `launch()`.


Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_mKgsNsOiQxp1vMOphoSHNaMu', function=Function(arguments='{"destination_city":"Sydney"}', name='get_ticket_price'), type='function')]))
DATABASE TOOL CALLED: Getting price for Sydney
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_3WfJxv4EaDdov9kz2SMYapeI', function=Function(arguments='{"destination_city":"Perth"}', name='get_ticket_price'), type='function')]))
DATABASE TOOL CALLED: Getting price for Perth
Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio